In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import copy
from multiprocessing import Pool
from IPython.display import clear_output
from IPython.display import HTML
import matplotlib.animation as animation
import matplotlib
from scipy.ndimage import gaussian_filter
matplotlib.use('macosx')

In [2]:
class Lenia:
    def __init__(self, n, m, kernel_outer_radius, kernel_inner_radius, smoothing_factor, iters, at, sigma, mu):

        self.iters = iters
        self.n = n
        self.m = m
        self.matrix = np.zeros((n, m))
        self.kernel = self.create_kernel(kernel_outer_radius, kernel_inner_radius, smoothing_factor=smoothing_factor)
        self.shape_kernel = len(self.kernel), len(self.kernel[0])
        self.half_size_i_kernel = int(self.shape_kernel[0] / 2)
        self.half_size_j_kernel = int(self.shape_kernel[1] / 2)
        self.sigma  = sigma
        self.mu = mu
        self.at = at

        # -- animation
        self.count_loop = 0
        self.animation = None

    def load_file(self, file):
        lista = []
        with open(file, 'r') as file:
            for line in file:
                lista.append(list(map(lambda e: float(e), line.replace('\n', '').split())))
        for i in range(len(lista)):
            self.matrix[int(lista[i][1])][int(lista[i][0])] = 1

    def load_points(self, points_x: list, points_y: list):
        if len(points_x) != len(points_y):
            raise Exception('Lists are not eaqual!')
        for i in range(len(points_x)):
            self.matrix[points_y[i]][points_x[i]] = 1

    # -------------------------------------
    # def create_kernel(self, outer_radius, inner_radius, smoothness):
    #     size = 2 * outer_radius + 1
    #     kernel = np.zeros((size, size))
    #     center = outer_radius
    # 
    #     for i in range(size):
    #         for j in range(size):
    #             distance = np.sqrt((i - center) ** 2 + (j - center) ** 2)
    #             if distance <= inner_radius:
    #                 kernel[i, j] = 0  # Inside the inner circle, values are 0
    #             elif distance <= outer_radius:
    #                 # Create a smooth transition from 0 to 1 using a smoothness factor
    #                 kernel[i, j] = (distance - inner_radius) / (outer_radius - inner_radius) ** smoothness
    # 
    #     return kernel
    
    def create_kernel(self, outer_radius, inner_radius, smoothing_factor):
        size = 2 * outer_radius + 1
        x, y = np.meshgrid(np.arange(size) - outer_radius, np.arange(size) - outer_radius)
        
        distance = np.sqrt(x**2 + y**2)
        
        # Create a binary ring mask by subtracting the inner circle from the outer circle
        outer_circle = (distance <= outer_radius).astype(float)
        inner_circle = (distance <= inner_radius).astype(float)
        ring = outer_circle - inner_circle
        
        # Apply Gaussian smoothing to the ring
        smoothed_ring = gaussian_filter(ring, sigma=smoothing_factor)
        
        return smoothed_ring

    def calc_U(self, matrix, i_c, j_c):
        u = 0
        count_k = np.sum(self.kernel)
        
        for i_k in range(self.shape_kernel[0]):
            for j_k in range(self.shape_kernel[1]):
                i_matrix_index = i_c - self.half_size_i_kernel + i_k
                j_matrix_index = j_c - self.half_size_j_kernel + j_k
                if (0 <= i_matrix_index < self.n) and (0 <= j_matrix_index < self.m):
                    u += matrix[i_matrix_index][j_matrix_index] * self.kernel[i_k][j_k]
                    # count_k += self.kernel[i_k][j_k]
    
        u = u / count_k
        return u
    
    def growth_func(self, u):
        l = abs(u - self.mu)
        k = 2 * (self.sigma ** 2)
        return 2 * np.exp(-(l ** 2) / k) - 1

    def calc_c_t(self, matrix, i, j):
        u = self.calc_U(matrix, i, j)
        a = self.growth_func(u)
        return np.clip((matrix[i][j] + self.at * a), 0, 1)
    
    def core_funct(self):
        fig = plt.figure(figsize=(4, 3))
        im = plt.imshow(self.matrix, cmap='magma', interpolation='nearest')
        plt.axis('off')

        def animation_loop(frame):
            matrix_tmp = np.zeros((self.n, self.m))
            for i in range(self.matrix.shape[0]):
                for j in range(self.matrix.shape[1]):
                    matrix_tmp[i][j] = self.calc_c_t(self.matrix, i, j)

            self.matrix = copy.deepcopy(matrix_tmp)
            im.set_array(self.matrix)
            plt.title(f'Generation: {self.count_loop}')
            self.count_loop += 1
            print(self.count_loop)
            return im,

        self.animation = animation.FuncAnimation(fig, func=animation_loop, frames=self.iters, interval=10,
                                                 cache_frame_data=False)
        # self.animation.save('lenia.gif')
        plt.show()


In [3]:
lenia = Lenia(n=120, m=120, kernel_outer_radius=7, kernel_inner_radius=3, iters=200, smoothing_factor=1, at=0.1, sigma=0.017, mu=0.115)
lenia.load_points(points_x=[random.randint(20, 60) for _ in range(1000)],
                  points_y=[random.randint(20, 60) for _ in range(1000)])
# lenia.load_file('data.dat')
lenia.core_funct()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
